Importing Libraries

In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../')

This notebooks gives examples on how to use the BayesianOptimization class.
It has built in funcionality for the following:
1. Suggesting more than 1 candidate ($q>1$) per iteration.
2. 3 different types of GP models (Single Task GP, Mixed Single Task GP, SAASBO (Model List GP for $d_{out}>1$)).
3. Optional one-hot encoding when not using Mixed Single Task GP.
4. Multi-Objective and Single-Objective optimization.
5. Weights for prioritizing targets.
6. Ingredient minimization with weights.
7. 3 different types of acquisition functions for Single-Objective optimization (UCB, EI, PI).
8. Expected Hypervolume Improvement for Multi-Objective optimization.
9. 2 different $q$-sampling strategies (Believer Update and Monte-Carlo). 

In [15]:
from bayesian_opt import BayesianOptimization
import numpy as np
import pandas as pd

Example usage of bayesian optimization for continuous data, where $d_\text{out}=1$. 

In [3]:
train_X = np.random.uniform(low = 0, high = 10, size = (10,4))
df = pd.DataFrame(train_X, columns=['pH','c1','c2', 'c3'])
df

,pH,c1,c2,c3
0,4.333185,0.231159,2.800266,5.328859
1,2.221875,7.673015,2.127829,6.730077
2,5.537705,1.034543,2.720425,7.420798
3,4.362857,1.661550,2.474850,8.428968
4,5.321095,2.344642,9.457541,4.002785
5,6.514101,6.947075,9.295819,4.949383
6,0.275523,8.682134,2.787865,5.031036
7,2.004624,0.710657,6.396303,8.180560
8,6.798207,1.703607,7.987494,7.910501
9,2.864952,2.724555,8.332451,1.066377


In [16]:
bo_model = BayesianOptimization().fit(df, ['c3'], model_type='Single-Task GP')

a = bo_model.candidates(1, export_df=True)
a

c:\Users\danin\AppData\Local\Programs\Python\Python313\Lib\site-packages\botorch\models\utils\assorted.py:264: InputDataWarning: Data (input features) is not contained to the unit cube. Please consider min-max scaling the input data.
  check_min_max_scaling(


,pH,c1,c2,c3
0,5.821353,1.193463,2.314354,6.527911


Example usage of bayesian optimization for mixed continuous and categorical data with target of dim 1 

In [17]:
train_X_cat = train_X
train_X_cat[:,0] = np.round(train_X_cat[:,0] )
df_cat = pd.DataFrame(train_X_cat, columns=['pH','c1','c2','c3'])
df_cat


,pH,c1,c2,c3
0,4.0,0.231159,2.800266,5.328859
1,2.0,7.673015,2.127829,6.730077
2,6.0,1.034543,2.720425,7.420798
3,4.0,1.661550,2.474850,8.428968
4,5.0,2.344642,9.457541,4.002785
5,7.0,6.947075,9.295819,4.949383
6,0.0,8.682134,2.787865,5.031036
7,2.0,0.710657,6.396303,8.180560
8,7.0,1.703607,7.987494,7.910501
9,3.0,2.724555,8.332451,1.066377


In [18]:
bo_model_mst = BayesianOptimization().fit(df_cat,['c3'],cat_dims=['pH'], model_type='Mixed Single-Task GP')

cand_1 = bo_model_mst.candidates(1, export_df=True)
cand_1

c:\Users\danin\AppData\Local\Programs\Python\Python313\Lib\site-packages\botorch\models\utils\assorted.py:264: InputDataWarning: Data (input features) is not contained to the unit cube. Please consider min-max scaling the input data.
  check_min_max_scaling(


,pH,c1,c2,c3
0,7.0,1.548939,2.518963,7.20412


In [19]:
bo_model_st = BayesianOptimization().fit(df_cat,['c3'],cat_dims=['pH'], model_type='Single-Task GP')

cand_2 = bo_model_st.candidates(1, export_df=True)
cand_2

c:\Users\danin\AppData\Local\Programs\Python\Python313\Lib\site-packages\botorch\models\utils\assorted.py:264: InputDataWarning: Data (input features) is not contained to the unit cube. Please consider min-max scaling the input data.
  check_min_max_scaling(


,pH,c1,c2,c3
0,4.0,1.559773,2.53962,7.820499


Choosing a different GP model type.

In [8]:
bo_model = BayesianOptimization().fit(df,['c3'],cat_dims=['pH'], model_type='SAASBO')

a = bo_model.candidates(2, export_df=True)
a

c:\Users\danin\AppData\Local\Programs\Python\Python313\Lib\site-packages\botorch\models\utils\assorted.py:264: InputDataWarning: Data (input features) is not contained to the unit cube. Please consider min-max scaling the input data.
  check_min_max_scaling(
c:\Users\danin\AppData\Local\Programs\Python\Python313\Lib\site-packages\botorch\models\utils\assorted.py:267: InputDataWarning: Data (outcome observations) is not standardized (std = tensor([2.2995], dtype=torch.float64), mean = tensor([5.9049], dtype=torch.float64)).Please consider scaling the input to zero mean and unit variance.
  check_standardization(Y=train_Y, raise_on_fail=raise_on_fail)


,pH,c1,c2,c3
0,7.0,1.365130,5.283355,8.523887
1,2.0,3.196111,2.427817,5.991574


Different $q$-sampling strategy.

In [20]:
bo_model = BayesianOptimization().fit(df,['c3'])

a = bo_model.candidates(4, q_sampling_method="Believer", export_df=True)
a

c:\Users\danin\AppData\Local\Programs\Python\Python313\Lib\site-packages\botorch\models\utils\assorted.py:264: InputDataWarning: Data (input features) is not contained to the unit cube. Please consider min-max scaling the input data.
  check_min_max_scaling(
c:\Users\danin\AppData\Local\Programs\Python\Python313\Lib\site-packages\botorch\models\utils\assorted.py:264: InputDataWarning: Data (input features) is not contained to the unit cube. Please consider min-max scaling the input data.
  check_min_max_scaling(
c:\Users\danin\AppData\Local\Programs\Python\Python313\Lib\site-packages\botorch\models\utils\assorted.py:264: InputDataWarning: Data (input features) is not contained to the unit cube. Please consider min-max scaling the input data.
  check_min_max_scaling(
c:\Users\danin\AppData\Local\Programs\Python\Python313\Lib\site-packages\botorch\models\utils\assorted.py:264: InputDataWarning: Data (input features) is not contained to the unit cube. Please consider min-max scaling the i

,pH,c1,c2,c3
0,6.735627,1.984701,7.954967,7.002646
1,6.729248,1.436465,8.076751,7.037620
2,3.904657,1.938033,2.532963,7.697008
3,1.869035,2.095339,5.289180,6.067858


Choosing a different acquisition function.

In [21]:
a = bo_model.candidates(4, export_df=True, acq_func_name="UCB")

Minimizing Ingredients

In [22]:
a = bo_model.candidates(4, q_sampling_method="Believer", export_df=True, input_weights={0:1.1, 1:2.5, 2:2.0})
a

c:\Users\danin\AppData\Local\Programs\Python\Python313\Lib\site-packages\botorch\models\utils\assorted.py:264: InputDataWarning: Data (input features) is not contained to the unit cube. Please consider min-max scaling the input data.
  check_min_max_scaling(
c:\Users\danin\AppData\Local\Programs\Python\Python313\Lib\site-packages\botorch\models\utils\assorted.py:264: InputDataWarning: Data (input features) is not contained to the unit cube. Please consider min-max scaling the input data.
  check_min_max_scaling(
c:\Users\danin\AppData\Local\Programs\Python\Python313\Lib\site-packages\botorch\models\utils\assorted.py:264: InputDataWarning: Data (input features) is not contained to the unit cube. Please consider min-max scaling the input data.
  check_min_max_scaling(


,pH,c1,c2,c3
0,4.195928,1.765453,2.263565,7.683453
1,1.844735,0.760223,6.084777,7.351392
2,2.047390,1.053458,6.426261,7.355018
3,4.030256,1.961834,2.484502,7.690757


Multi-Objective optimization.

In [23]:
bo_model = BayesianOptimization().fit(df,['c2', 'c3'])

a = bo_model.candidates(4, export_df=True)
a

c:\Users\danin\AppData\Local\Programs\Python\Python313\Lib\site-packages\botorch\models\utils\assorted.py:264: InputDataWarning: Data (input features) is not contained to the unit cube. Please consider min-max scaling the input data.
  check_min_max_scaling(
c:\Users\danin\AppData\Local\Programs\Python\Python313\Lib\site-packages\botorch\acquisition\multi_objective\monte_carlo.py:111: NumericsWarning: qExpectedHypervolumeImprovement has known numerical issues that lead to suboptimal optimization performance. It is strongly recommended to simply replace

	 qExpectedHypervolumeImprovement 	 --> 	 qLogExpectedHypervolumeImprovement 

instead, which fixes the issues and has the same API. See https://arxiv.org/abs/2310.20708 for details.
  legacy_ei_numerics_warning(legacy_name=type(self).__name__)


,pH,c1,c2,c3
0,2.000000,0.710658,6.389890,8.165329
1,5.000007,2.344665,9.430636,4.015518
2,0.003825,0.231602,5.438084,5.904934
3,3.422795,5.698747,5.438084,5.904934
